In [1]:
#Alper Beykan YOLCU

In [3]:
import cv2
import numpy as np
import random


from copy import deepcopy


In [4]:
def get_image(path):
    im=  cv2.imread(path)
    return im


In [5]:
def show_im(im):
    cv2.imshow("",im)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [6]:
def cut(point, homography):
    point = np.array([[point[0]], [point[1]], [1]])
    cuted_dest = np.dot(homography, point)

    if cuted_dest[2] is not 0:
        cuted_dest = cuted_dest / cuted_dest[2]

    return np.array([cuted_dest[0][0], cuted_dest[1][0]])


<>:5: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:5: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-6-c2adb455f0b2>:5: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if cuted_dest[2] is not 0:


In [7]:

def get_kp(image, file_name=None):
    orb = cv2.ORB_create()
    image1_kp, image1_descriptor = orb.detectAndCompute(image, None)

    return [image1_kp, image1_descriptor]

def fetch_matches(image1_details, image2_details):
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(image1_details[1], image2_details[1])
    matches = sorted(matches, key=lambda x: x.distance)
    return matches


In [8]:
def compute_inlier_count(homography, image1_keypoints, image2_keypoints, matches, inlier_threshold):

    src_pts = np.array([image1_keypoints[match.queryIdx].pt for match in matches], dtype=np.int)
    dest_pts = np.array([image2_keypoints[match.trainIdx].pt for match in matches], dtype=np.int)

    inlier_elements = []
    for index in range(0, len(src_pts)):
        src = src_pts[index]
        dest = dest_pts[index]
        cuted_dest = cut(src, homography)

        ssd = np.sqrt((np.array(cuted_dest - dest, dtype=np.float32) ** 2).sum(0))
        if ssd < inlier_threshold:
            inlier_elements.append(index)

    return inlier_elements

def ransac_matches(inmatches, image1_keypoints, image2_keypoints):
    src_pts = np.array([image1_keypoints[match.queryIdx].pt for match in inmatches], dtype=np.int)
    dst_pts = np.array([image2_keypoints[match.trainIdx].pt for match in inmatches], dtype=np.int)
    homography, mask = cv2.findHomography(src_pts, dst_pts, method=0)
    return homography

def ransac(matches, num_matches, num_iterations, image1_keypoints, image2_keypoints, inlier_threshold):
    max_inlier_elements = []

    for index in range(0, num_iterations):
        random_4_matches = random.sample(matches, num_matches)
        src_pts = np.array([image1_keypoints[match.queryIdx].pt for match in random_4_matches], dtype=np.int)
        dst_pts = np.array([image2_keypoints[match.trainIdx].pt for match in random_4_matches], dtype=np.int)
        homography, mask = cv2.findHomography(src_pts, dst_pts, method=0)

        inlier_elements = compute_inlier_count(homography, image1_keypoints, image2_keypoints, matches, inlier_threshold)
        if len(inlier_elements) >= len(max_inlier_elements):
            max_inlier_elements = inlier_elements

    inmatches = []
    for index in max_inlier_elements:
        inmatches.append(matches[index])

    homography = ransac_matches(inmatches, image1_keypoints, image2_keypoints)
    inv_homography = np.linalg.inv(homography)

    return homography, inv_homography, inmatches


In [9]:

def draw_kp(image, corner_points):

    image = deepcopy(image)
    for corner_point in corner_points:
        x, y = corner_point.pt
        cv2.circle(image, (int(x), int(y)), 4, (0, 0, 255))

    return image

In [10]:

def calculate_stitched_image_bounds(image1, image2, inv_homography):
    image1_height, image1_width, x = image1.shape
    image2_height, image2_width, x = image2.shape
    bottom_left = cut([0, image2_height - 1], inv_homography)
    bottom_right = cut([image2_width - 1, image2_height - 1], inv_homography)

    top_left = cut([0, 0], inv_homography)
    top_right = cut([image2_width - 1, 0], inv_homography)
   
    stitchtop = int(min(top_left[1], top_right[1], 0))
    stitchbottom = int(max(bottom_left[1], bottom_right[1], image1_height))
    stitchleft = int(min(top_left[0], bottom_left[0], 0))
    stitchright = int(max(top_right[0], bottom_right[0], image1_width))
    width = stitchright - stitchleft

    height = stitchbottom - stitchtop

    offset = (-stitchleft, -stitchtop)

    min_x = int(min(top_left[0], bottom_left[0]))
    min_y = int(min(top_left[1], top_right[1]))
    min_xy = (abs(min_x), abs(min_y))

    stitched_image_bounds = (height, width, 3)
    return stitched_image_bounds, offset, min_xy

def stitch(image1, image2, homography, inv_homography):

    stitched_image_bounds, offset, min_xy = calculate_stitched_image_bounds(image1, image2, inv_homography)
    image1_height, image1_width, x = image1.shape
  
    (offset_x, offset_y) = offset
    translation = np.matrix([
        [1.0, 0.0, offset_x],
        [0, 1.0, offset_y],
        [0.0, 0.0, 1.0]
    ])

    inv_homography = translation * inv_homography

    (bounds_x, bounds_y, z) = stitched_image_bounds

    stitched_image = np.zeros(shape=stitched_image_bounds, dtype=np.uint8)
    stitched_image = cv2.warpPerspective(image2, inv_homography, (bounds_y, bounds_x), stitched_image)
    stitched_image[offset_y:image1_height + offset_y, offset_x:image1_width + offset_x] = image1


    return stitched_image

In [22]:

left=get_image("C:/Users/user/Desktop/dersler/imageProcessing/CENG471 Assignment 3/left.jpeg")
right=get_image("C:/Users/user/Desktop/dersler/imageProcessing/CENG471 Assignment 3/right.jpeg")
image1 = left
image2 = right

image1_details = get_kp(image1, "im1") 
image2_details = get_kp(image2, "im2")

matches = fetch_matches(image1_details, image2_details)
matches_image = cv2.drawMatches(image1, image1_details[0], image2, image2_details[0], matches[:20], None, flags=2)
show_im(matches_image)
homography, inv_homography, inlier_matches = ransac(matches, 4, 100, image1_details[0], image2_details[0], inlier_threshold=0.5)
inlier_image = cv2.drawMatches(image1, image1_details[0], image2, image2_details[0], inlier_matches, None, flags=2)
stitched_image = stitch(image1, image2, homography, inv_homography)
show_im(stitched_image)
cv2.waitKey(0)
print("Done")

Done


In [23]:
import os

directory = r'C:/Users/user/Desktop/dersler/imageProcessing/CENG471 Assignment 3/'
  
print("Before saving image:")   
print(os.listdir(directory))   
  
# Filename 
filename = 'result.jpg'
cv2.imwrite(filename, stitched_image) 
filename = 'result2.jpg'
  
cv2.imwrite(filename, matches_image) 
# List files and directories   
# in 'C:/Users / Rajnish / Desktop / GeeksforGeeks'   
print("After saving image:")   
print(os.listdir(directory)) 
  

Before saving image:
['.ipynb_checkpoints', '16118072_Ceng471_hw3.ipynb', 'Assignment3.pdf', 'left.jpeg', 'result.jpg', 'result2.jpg', 'right.jpeg']
After saving image:
['.ipynb_checkpoints', '16118072_Ceng471_hw3.ipynb', 'Assignment3.pdf', 'left.jpeg', 'result.jpg', 'result2.jpg', 'right.jpeg']
